# Backtester V1

Indicators in use:
- EMA20
- EMA50
- CPR

### 1. Load the OHLC data and calculate the indicators.
#### 1.1. Calculation of EMA20 and EMA50.

In [1]:
# import Libraries
import pandas as pd
import datetime as dt
import numpy as np
import itertools

ema_period_1 = 20
ema_period_2 = 50

# Load all the data needed: NIFTY Index Data
stock_data = pd.read_csv("^NSEI-5-Minutes-DATA.csv")

# Calculate EMA20
smoothing = 2 / (ema_period_1 + 1)
# Calculate the first EMA value and assign it to the DataFrame
stock_data[f"EMA{ema_period_1}"] = stock_data["Close"].ewm(span = ema_period_1, adjust = True, min_periods = 0).mean()
# Round off the values
stock_data[f"EMA{ema_period_1}"] = stock_data[f"EMA{ema_period_1}"].round(2)


# Calculate EMA50
smoothing = 2 / (ema_period_2 + 1)
# Calculate the first EMA value and assign it to the DataFrame
stock_data[f"EMA{ema_period_2}"] = stock_data["Close"].ewm(span = ema_period_2, adjust = True, min_periods = 0).mean()
# Round off the values
stock_data[f"EMA{ema_period_2}"] = stock_data[f"EMA{ema_period_2}"].round(2)

stock_data

,Date,Time,Open,High,Low,Close,EMA20,EMA50
0,2015-01-09,14:30:00,8240.00,8246.75,8230.65,8237.30,8237.30,8237.30
1,2015-01-09,14:35:00,8237.35,8253.15,8237.10,8252.55,8245.31,8245.08
2,2015-01-09,14:40:00,8252.35,8257.30,8246.25,8252.55,8247.97,8247.67
3,2015-01-09,14:45:00,8252.40,8261.90,8249.30,8258.30,8250.95,8250.49
4,2015-01-09,14:50:00,8258.20,8280.70,8258.20,8275.95,8257.00,8256.00
...,...,...,...,...,...,...,...,...
150246,2023-07-27,15:05:00,19631.60,19652.40,19631.60,19650.30,19661.76,19702.27
150247,2023-07-27,15:10:00,19649.45,19665.95,19645.45,19664.70,19662.04,19700.79
150248,2023-07-27,15:15:00,19664.65,19678.75,19657.20,19677.05,19663.47,19699.86
150249,2023-07-27,15:20:00,19677.35,19697.95,19666.80,19695.65,19666.53,19699.70


In [2]:
date_groups = stock_data.groupby("Date")

# Create empty arrays for value storage
date = np.array([])
day_open = np.array([])
day_close = np.array([])
day_high = np.array([])
day_low = np.array([])

# Get the dates, day opening, day high, day low and closing values
for key, group in date_groups:
    date = np.append(date, key)
    day_open = np.append(day_open, group["Open"].iloc[0])
    day_close = np.append(day_close, group["Close"].iloc[-1])
    day_high = np.append(day_high, group["High"].max())
    day_low = np.append(day_low, group["Low"].min())

#### 1.2. Calculation of CPR levels.
The formula used to calculate CPR is standard. It is mentioned on Zerodha's website [here](https://support.zerodha.com/category/trading-and-markets/kite-web-and-mobile/charts/articles/how-are-pivot-points-derived-while-using-the-cpr).

The first row of CPR levels is "NaN" as CPR formula uses values from yesterday's Day OHLC.

In [3]:
daily_values = {
    "Date": date,
    "Day Open": day_open,
    "Day High": day_high,
    "Day Low": day_low,
    "Day Close": day_close
    }

cpr_df = pd.DataFrame(daily_values)

cpr_df[["PP", "BCP", "TCP", "S1", "S2", "S3", "R1", "R2", "R3"]] = None

# Calculate CPR Levels using shift fn. (Faster compared to standard Loop)
cpr_df["PP"] = ((cpr_df["Day High"].shift() + cpr_df["Day Low"].shift() + cpr_df["Day Close"].shift()) / 3).round(2)
cpr_df["BCP"] = ((cpr_df["Day High"].shift() + cpr_df["Day Low"].shift()) / 2).round(2)
cpr_df["TCP"] = ((cpr_df["PP"] - cpr_df["BCP"]) + cpr_df["PP"]).round(2)
cpr_df["S1"] = ((2 * cpr_df["PP"]) - cpr_df["Day High"].shift()).round(2)
cpr_df["S2"] = (cpr_df["PP"] - (cpr_df["Day High"].shift() - cpr_df["Day Low"].shift())).round(2)
cpr_df["S3"] = (cpr_df["PP"] - 2 * (cpr_df["Day High"].shift() - cpr_df["Day Low"].shift())).round(2)
cpr_df["R1"] = ((2 * cpr_df["PP"]) - cpr_df["Day Low"].shift()).round(2)
cpr_df["R2"] = (cpr_df["PP"] + (cpr_df["Day High"].shift() - cpr_df["Day Low"].shift())).round(2)
cpr_df["R3"] = (cpr_df["PP"] + 2 * (cpr_df["Day High"].shift() - cpr_df["Day Low"].shift())).round(2)

cpr_df

,Date,Day Open,Day High,Day Low,Day Close,PP,BCP,TCP,S1,S2,S3,R1,R2,R3
0,2015-01-09,8240.00,8296.35,8230.65,8280.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-12,8291.35,8332.25,8245.60,8326.55,8269.08,8263.50,8274.66,8241.81,8203.38,8137.68,8307.51,8334.78,8400.48
2,2015-01-13,8346.15,8356.65,8268.15,8303.80,8301.47,8288.92,8314.02,8270.69,8214.82,8128.17,8357.34,8388.12,8474.77
3,2015-01-14,8307.25,8326.30,8236.75,8287.75,8309.53,8312.40,8306.66,8262.41,8221.03,8132.53,8350.91,8398.03,8486.53
4,2015-01-15,8425.20,8526.90,8380.75,8461.15,8283.60,8281.52,8285.68,8240.90,8194.05,8104.50,8330.45,8373.15,8462.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2006,2023-07-21,19800.45,19886.35,19700.10,19780.90,19906.40,19874.62,19938.18,19822.00,19674.05,19441.70,20054.35,20138.75,20371.10
2007,2023-07-24,19748.45,19782.15,19659.60,19665.40,19789.12,19793.22,19785.02,19691.89,19602.87,19416.62,19878.14,19975.37,20161.62
2008,2023-07-25,19729.35,19729.35,19616.65,19678.85,19702.38,19720.88,19683.88,19622.61,19579.83,19457.28,19745.16,19824.93,19947.48
2009,2023-07-26,19733.35,19825.10,19716.75,19773.10,19674.95,19673.00,19676.90,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35


### 2. Merge the EMA's, CPR levels & Day OHLC into Main dataset. Group the data on the basis of date.
#### 2.1. Merging the data.

In [4]:
stock_data["Date"] = pd.to_datetime(stock_data["Date"])
cpr_df["Date"] = pd.to_datetime(cpr_df["Date"])

merger = stock_data.merge(cpr_df, on = "Date", how = "left") 
trading_data = merger.fillna(0)

pd.set_option("display.max_columns", None)
trading_data

,Date,Time,Open,High,Low,Close,EMA20,EMA50,Day Open,Day High,Day Low,Day Close,PP,BCP,TCP,S1,S2,S3,R1,R2,R3
0,2015-01-09,14:30:00,8240.00,8246.75,8230.65,8237.30,8237.30,8237.30,8240.0,8296.35,8230.65,8280.25,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00
1,2015-01-09,14:35:00,8237.35,8253.15,8237.10,8252.55,8245.31,8245.08,8240.0,8296.35,8230.65,8280.25,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00
2,2015-01-09,14:40:00,8252.35,8257.30,8246.25,8252.55,8247.97,8247.67,8240.0,8296.35,8230.65,8280.25,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00
3,2015-01-09,14:45:00,8252.40,8261.90,8249.30,8258.30,8250.95,8250.49,8240.0,8296.35,8230.65,8280.25,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00
4,2015-01-09,14:50:00,8258.20,8280.70,8258.20,8275.95,8257.00,8256.00,8240.0,8296.35,8230.65,8280.25,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150246,2023-07-27,15:05:00,19631.60,19652.40,19631.60,19650.30,19661.76,19702.27,19850.9,19867.50,19603.85,19700.20,19771.65,19770.92,19772.38,19718.2,19663.3,19554.95,19826.55,19880.0,19988.35
150247,2023-07-27,15:10:00,19649.45,19665.95,19645.45,19664.70,19662.04,19700.79,19850.9,19867.50,19603.85,19700.20,19771.65,19770.92,19772.38,19718.2,19663.3,19554.95,19826.55,19880.0,19988.35
150248,2023-07-27,15:15:00,19664.65,19678.75,19657.20,19677.05,19663.47,19699.86,19850.9,19867.50,19603.85,19700.20,19771.65,19770.92,19772.38,19718.2,19663.3,19554.95,19826.55,19880.0,19988.35
150249,2023-07-27,15:20:00,19677.35,19697.95,19666.80,19695.65,19666.53,19699.70,19850.9,19867.50,19603.85,19700.20,19771.65,19770.92,19772.38,19718.2,19663.3,19554.95,19826.55,19880.0,19988.35


#### 2.2. Grouping the data on the basis of date.

In [5]:
# Create groups according to dates for intraday
date_groups = trading_data.groupby("Date")

# This is used to test the strategy before using it for the complete dataset.
test_group = date_groups.get_group(list(date_groups.groups.keys())[2009])
test_group

,Date,Time,Open,High,Low,Close,EMA20,EMA50,Day Open,Day High,Day Low,Day Close,PP,BCP,TCP,S1,S2,S3,R1,R2,R3
150101,2023-07-26,09:15:00,19733.35,19744.80,19716.75,19724.80,19683.56,19679.61,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35
150102,2023-07-26,09:20:00,19724.35,19750.45,19722.65,19748.70,19689.77,19682.32,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35
150103,2023-07-26,09:25:00,19748.35,19762.25,19745.50,19753.90,19695.87,19685.12,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35
150104,2023-07-26,09:30:00,19753.35,19761.00,19747.15,19756.80,19701.68,19687.93,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35
150105,2023-07-26,09:35:00,19756.85,19765.70,19751.30,19753.05,19706.57,19690.49,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150171,2023-07-26,15:05:00,19779.00,19779.10,19770.05,19775.90,19789.33,19788.19,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35
150172,2023-07-26,15:10:00,19775.95,19784.75,19773.75,19784.70,19788.89,19788.06,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35
150173,2023-07-26,15:15:00,19784.35,19785.75,19776.90,19779.60,19788.01,19787.73,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35
150174,2023-07-26,15:20:00,19779.30,19784.40,19777.35,19778.15,19787.07,19787.35,19733.35,19825.1,19716.75,19773.1,19674.95,19673.0,19676.9,19620.55,19562.25,19449.55,19733.25,19787.65,19900.35


### 3. CPR Level Combinations.
CPR levels can be used as supports and resistances. Combination of CPR levels can be made on the basis of trade direction.
Example: S1 -> PP corresponds to a successful LONG trade and vice versa.

This is done to find out on which levels does the stock perform well as per strategy's conditions.

#### 3.1 Creating permutations of levels.
##### Short side permutations.

In [6]:
# Creating all the possible combinations for taking trades between CPR levels
levels = ["S1", "S2", "S3", "PP", "R1", "R2", "R3"]

# Using itertools module's permutation function
permutations = list(itertools.permutations(levels, r = 2))

parts = len(permutations) // 7

all_list = []
sell_side_combos = []
buy_side_combos = []

# Creating list of permutations. 
for i in range(0, len(permutations), parts):
    single_list = permutations[i: i + parts]
    all_list.append(single_list)

# Storing all lists for manually getting the SHORT side permutations.
first = all_list[0]
second = all_list[1]
third = all_list[2]
fourth = all_list[3]
fifth = all_list[4]
sixth = all_list[5]
seventh = all_list[6]


sell_side_combos.extend(first[:2])
sell_side_combos.append(second[1])
sell_side_combos.extend(fourth[:3])
sell_side_combos.extend(fifth[:4])
sell_side_combos.extend(sixth[:5])
sell_side_combos.extend(seventh)

# All the SHORT side permutations
sell_side_combos

[('S1', 'S2'),
 ('S1', 'S3'),
 ('S2', 'S3'),
 ('PP', 'S1'),
 ('PP', 'S2'),
 ('PP', 'S3'),
 ('R1', 'S1'),
 ('R1', 'S2'),
 ('R1', 'S3'),
 ('R1', 'PP'),
 ('R2', 'S1'),
 ('R2', 'S2'),
 ('R2', 'S3'),
 ('R2', 'PP'),
 ('R2', 'R1'),
 ('R3', 'S1'),
 ('R3', 'S2'),
 ('R3', 'S3'),
 ('R3', 'PP'),
 ('R3', 'R1'),
 ('R3', 'R2')]

##### Long side combinations.

In [7]:
sample_space = []
buy_side_combos = []

for lists in all_list:
    for item in lists:
        sample_space.append(item)

for item in sample_space:
    if item not in sell_side_combos:
        buy_side_combos.append(item)

# Only took complementary of sell_side_combos
buy_side_combos

[('S1', 'PP'),
 ('S1', 'R1'),
 ('S1', 'R2'),
 ('S1', 'R3'),
 ('S2', 'S1'),
 ('S2', 'PP'),
 ('S2', 'R1'),
 ('S2', 'R2'),
 ('S2', 'R3'),
 ('S3', 'S1'),
 ('S3', 'S2'),
 ('S3', 'PP'),
 ('S3', 'R1'),
 ('S3', 'R2'),
 ('S3', 'R3'),
 ('PP', 'R1'),
 ('PP', 'R2'),
 ('PP', 'R3'),
 ('R1', 'R2'),
 ('R1', 'R3'),
 ('R2', 'R3')]

### 4. Check for CPR level permutations.
#### 4.1. Long side check.
finding out on which days stock price hit the first level and the second level. This will tell us which permutations of CPR levels are crossed in the correct direction. Later on, these levels will be prioritized.

In [8]:
# Storing the no. of times stock crossed level 1 and level 2.
# Count = 1 if stock crosses both levels in desired direction atleast once.

# Storing the permutations for creating a DataFrame later.
buy_counts = []
buy_combinations = []

# Takes permutations and inserts them at x[0] and x[1].
# x[0] Lower level & x[1] Higher level.
for i in range(len(buy_side_combos)):
    x = buy_side_combos[i]
    buy_combinations.append(x)

    all_count = 0

    # Loop through everyday  and find if stock price crosses level 2 after crossing or touching level 1.
    for date, group in date_groups:

        all_candles = group["High"] 

        for i in range(len(group)):

            if group["Low"].iloc[i] < group[f"{x[0]}"].iloc[i] < group["High"].iloc[i]:

                # If yes, then counter = 1 and move on to the next day.
                if np.any(all_candles[i:] > group[f"{x[1]}"].iloc[0]):
                    counter = 1
                    break
            else:
                counter = 0
        
        # Gives the count of days when level 2 was broken after stock price touched level 1.
        all_count = all_count + counter

    buy_counts.append(all_count)

#### 4.2. Short side check.

In [9]:
# Same logic for short side.
sell_counts = []
sell_combinations = []

for i in range(len(sell_side_combos)):
    x = sell_side_combos[i]
    sell_combinations.append(x)

    all_count = 0

    for date, group in date_groups:

        all_candles = group["Low"] 

        for i in range(len(group)):

            if group["High"].iloc[i] > group[f"{x[0]}"].iloc[i] > group["Low"].iloc[i]:

                if np.any(all_candles[i:] < group[f"{x[1]}"].iloc[0]):
                    counter = 1
                    break
            else:
                counter = 0
            
        all_count = all_count + counter

    sell_counts.append(all_count)

### 5. Getting the results.
Checking the permutations of CPR levels where level 2 was broken after stock price touched level 1.
#### 5.1. Creating DataFrames for Long and Short side trades.

In [10]:
# For Long side
cpr_buy_stats = {
    "BUY Combos": buy_combinations,
    "BUY Count": buy_counts
}

# For Short side
cpr_sell_stats = {
    "SELL Combos": sell_combinations,
    "SELL Count": sell_counts
}

# Merging both 
cpr_buy_df = pd.DataFrame(cpr_buy_stats).sort_values(by = "BUY Count", ascending = False, ignore_index = True)
cpr_sell_df = pd.DataFrame(cpr_sell_stats).sort_values(by = "SELL Count", ascending = False, ignore_index = True)

cpr_stats = pd.merge(cpr_buy_df, cpr_sell_df, left_index = True, right_index = True)
cpr_stats

,BUY Combos,BUY Count,SELL Combos,SELL Count
0,"(PP, R1)",380,"(PP, S1)",476
1,"(R1, R2)",326,"(S1, S2)",366
2,"(S1, PP)",290,"(R1, PP)",352
3,"(S2, S1)",170,"(PP, S2)",206
4,"(PP, R2)",107,"(R2, R1)",166
5,"(S1, R1)",77,"(R1, S1)",162
6,"(S2, PP)",62,"(S2, S3)",79
7,"(R2, R3)",48,"(R2, PP)",74
8,"(S1, R2)",27,"(R1, S2)",73
9,"(S2, R1)",26,"(S1, S3)",57


#### 5.2. Considered levels for strategy.
Selecting permutations having count > 170. I.e, permutation must satisfy the Short and Long side crossing atleast 10% of all the days.
Total days = 2011.
These levels will be prioritized for further strategies.

In [11]:
consd_cpr_buy_df = cpr_buy_df[cpr_buy_df["BUY Count"] >= 170]
consd_cpr_sell_df = cpr_sell_df[cpr_sell_df["SELL Count"] >=  170]

considered_cpr_levels = pd.merge(consd_cpr_sell_df, consd_cpr_buy_df, left_index = True, right_index = True)
considered_cpr_levels

,SELL Combos,SELL Count,BUY Combos,BUY Count
0,"(PP, S1)",476,"(PP, R1)",380
1,"(S1, S2)",366,"(R1, R2)",326
2,"(R1, PP)",352,"(S1, PP)",290
3,"(PP, S2)",206,"(S2, S1)",170


### 6. Main strategy.
We have to create a Wallet and decide position sizing for our strategy.
Following are the assumptions taken for the strategy:
- Derivatives (FUTURES) Intraday Trading

- Starting Balance = 10,00,000 INR

- Deployed Amount = 10% of Total Balance

- Futures contract = 9% of Index Value 


#### 6.1. Long Side
This strategy is using CPR levels. Different strategies can be developed and combined together to get desired results.

In [136]:
# Define Wallet
buy_wallet = 1000000

# Lot Size
lotsize = 50 

# Create list for storing important values
dates_buy = np.array([])
time_buy = np.array([])
entry_buy = np.array([])
exit_buy = np.array([])
sl_array_buy = np.array([])
net_rr_buy = np.array([])
buy_earnings = np.array([])

# BUY SIDE
for j in range(len(considered_cpr_levels["BUY Combos"])):
    x = considered_cpr_levels["BUY Combos"][j]

    for date, group in date_groups:

        group = group.reset_index(drop = True)

        all_candles_high = group["High"].to_numpy()
        all_candles_low = group["Low"].to_numpy()
        all_candles_open = group["Open"].to_numpy()
        all_candles_close = group["Close"].to_numpy()
        day_high = group["Day High"].to_numpy()
        day_low = group["Day Low"].to_numpy()
        all_candles_time = group["Time"].to_numpy()

        # Loop through the group
        for i in range(len(group)):      

# 1st BUY Condition            
        # Hammer Candle: If Opening of candle is above 30% and upper wick is max 20% of complete candle, Consider it a hammer candle.
            hammer_candle_1 = (all_candles_open[i] - all_candles_low[i]) >= ((all_candles_high[i] - all_candles_low[i]) * 0.22)
            hammer_candle_2 = (all_candles_high[i] - all_candles_close[i]) <= ((all_candles_high[i] - all_candles_low[i]) * 0.12)
            hammer_candle_3 = all_candles_low[i] <= group["R1"].iloc[0] <= all_candles_open[i]
            current_hammer_candle = ((hammer_candle_1 and hammer_candle_2) and hammer_candle_3)

            if current_hammer_candle:

                dates_buy = np.append(dates_buy, group["Date"].iloc[0])
                time_buy = np.append(time_buy, all_candles_time[i])
                entry_buy = np.append(entry_buy, all_candles_high[i])

                for z in range(1, 100):
                    if z * ((all_candles_high[i] * lotsize) / 9) > 1.0 * buy_wallet:
                        no_of_lots = z - 1
                        deployed_capital = no_of_lots * ((all_candles_high[i] * lotsize) / 9)
                        break

                buy_wallet -= deployed_capital  
            
                # StopLoss at candle LOW, GAP = 0.01%
                sl = (all_candles_high[i] - (all_candles_high[i] * 0.0029)).round(2)
                sl_array_buy = np.append(sl_array_buy, sl)   
                nishana = (all_candles_high[i] + (all_candles_high[i] * 0.0057)).round(2)  

                risk = all_candles_high[i] - sl
                reward = nishana - all_candles_high[i]
                rr_buy = (reward / risk).round(2) 

                net_rr_buy = np.append(net_rr_buy, rr_buy)

            # Exit at Target, SL or Day close
                for k in range(i, len(group)):
                    if all_candles_high[k] >= nishana: 
                        exit_val = nishana
                        break
                    if all_candles_low[k] <= sl:
                        exit_val = sl
                        break
                    else:
                        exit_val = all_candles_close[-1]
                exit_buy = np.append(exit_buy, exit_val)
            
            # Append the earnings
                earning_buy = deployed_capital + ((exit_val - all_candles_high[i]) * lotsize * no_of_lots)
                buy_wallet = buy_wallet + earning_buy
                buy_earnings = np.append(buy_earnings, buy_wallet)

                
...
# N Conditions can be added here or seperately
# EMA20 and EMA50 will be used in the strategy later on.


backtesting_long_data = {
    "Date": dates_buy,
    "Time": time_buy,
    "Entry": entry_buy,
    "SL": sl_array_buy,
    "Exit": exit_buy,
    "RR": net_rr_buy,
    "P&L": None,
    "Earnings": buy_earnings,
    }

backtesting_buy_df = pd.DataFrame(backtesting_long_data)

In [137]:
backtesting_buy_df["P&L"] = (backtesting_buy_df["Exit"] - backtesting_buy_df["Entry"]).round(2)

backtesting_buy_df

,Date,Time,Entry,SL,Exit,RR,P&L,Earnings
0,2015-02-10,12:30:00,8583.25,8558.36,8558.36,1.97,-24.89,975110.0
1,2015-02-18,11:15:00,8857.70,8832.01,8865.25,1.97,7.55,982282.5
2,2015-02-19,15:15:00,8911.80,8885.96,8898.35,1.97,-13.45,969505.0
3,2015-02-28,15:15:00,8911.25,8885.41,8920.75,1.97,9.50,978530.0
4,2015-05-15,10:05:00,8260.90,8236.94,8236.94,1.97,-23.96,953372.0
...,...,...,...,...,...,...,...,...
1055,2023-06-16,09:50:00,18771.85,18717.41,18823.60,1.97,51.75,2017965.0
1056,2023-07-06,10:20:00,19449.70,19393.30,19486.25,1.97,36.55,2050860.0
1057,2023-07-06,12:50:00,19455.45,19399.03,19486.25,1.97,30.80,2078580.0
1058,2023-07-11,09:15:00,19450.30,19393.89,19448.15,1.97,-2.15,2076537.5


Long side with 1 Strategy: Gross P&L = 86052 INR from 2015 Jan - 2023 Jul.

In [147]:
# Defining Wallet
sell_wallet = 1000000

# Create list for storing important values
dates_sell = np.array([])
time_sell = np.array([])
entry_sell = np.array([])
exit_sell = np.array([])
sl_array_sell = np.array([])
net_rr_sell = np.array([])
sell_earnings = np.array([])

# Lot Size
lotsize = 50

# SELL SIDE
for l in range(len(considered_cpr_levels["SELL Combos"])):
    y = considered_cpr_levels["SELL Combos"][l]

    for date, group in date_groups:

        group = group.reset_index(drop = True)

        all_candles_high = group["High"].to_numpy()
        all_candles_low = group["Low"].to_numpy()
        all_candles_open = group["Open"].to_numpy()
        all_candles_close = group["Close"].to_numpy()
        day_high = group["Day High"].to_numpy()
        day_low = group["Day Low"].to_numpy()
        all_candles_time = group["Time"].to_numpy()

        # Loop through the group
        for i in range(len(group)):

# 1st SELL Condition
        # Single Candle Entry: Candle crosses level. length of candle = 0.04% of candle close value
            # single_candle_1 = all_candles_open[i] > group[f"{y[0]}"].iloc[0] > all_candles_close[i]
            # single_candle_2 = (0.0007 * all_candles_close[i]) >= (all_candles_open[i] - all_candles_close[i]) >= (0.0003 * all_candles_close[i])
            # single_candle_position = (group[f"{y[0]}"].iloc[0] - all_candles_close[i]) >= (all_candles_open[i] - all_candles_close[i]) * 0.262

            # if (single_candle_1 and single_candle_2) and single_candle_position:

            #     dates_sell = np.append(dates_sell, group["Date"].iloc[0])
            #     time_sell = np.append(time_sell, all_candles_time[i])
            #     entry_sell = np.append(entry_sell, all_candles_low[i])  

            #     for z in range(1, 100):
            #         if z * ((all_candles_low[i] * lotsize) / 9) > 1.0 * buy_wallet:
            #             no_of_lots = z - 1
            #             deployed_capital_1 = no_of_lots * ((all_candles_low[i] * lotsize) / 9)
            #             break
            #     sell_wallet -= deployed_capital_1 

            # # StopLoss at candle LOW, GAP = 0.01%
            #     sl = (all_candles_low[i] + (all_candles_low[i] * 0.0031)).round(2)
            #     sl_array_sell = np.append(sl_array_sell, sl)  
            #     nishana = (all_candles_low[i] - (all_candles_low[i] * 0.00611)).round(2)
                
            #     risk = sl - all_candles_low[i]
            #     reward = all_candles_low[i] - nishana
            #     rr_sell = (reward / risk).round(2) 
            #     net_rr_sell = np.append(net_rr_sell, rr_sell)

            # # Exit at Target, SL or Day close
            #     for k in range(i, len(group)):
            #         if all_candles_low[k] <= nishana: 
            #             exit_val = nishana
            #             break
            #         if all_candles_high[k] >= sl:
            #             exit_val = sl
            #             break
            #         else:
            #             exit_val = all_candles_close[-1]
            #     exit_sell = np.append(exit_sell, exit_val)
            
            # # Append the earning
            #     earning_sell = deployed_capital_1 + ((all_candles_low[i] - exit_val) * lotsize * no_of_lots)
            #     sell_wallet = sell_wallet + earning_sell
            #     sell_earnings = np.append(sell_earnings, sell_wallet)


# 2nd SELL Condition
        # Hammer Candle: If Opening of candle is above 30% and upper wick is max 20% of complete candle, Consider it a hammer candle.
            hammer_candle_1 = (all_candles_high[i] - all_candles_open[i]) >= ((all_candles_high[i] - all_candles_low[i]) * 0.20)
            hammer_candle_2 = (all_candles_close[i] - all_candles_low[i]) <= ((all_candles_high[i] - all_candles_low[i]) * 0.124)
            hammer_candle_3 = all_candles_open[i] <= group[f"{y[0]}"].iloc[0] <= all_candles_high[i]
            current_hammer_candle = ((hammer_candle_1 and hammer_candle_2) and hammer_candle_3)

            if current_hammer_candle:

                dates_sell = np.append(dates_sell, group["Date"].iloc[0])
                time_sell = np.append(time_sell, all_candles_time[i])
                entry_sell = np.append(entry_sell, all_candles_low[i])  

                for z in range(1, 100):
                    if z * ((all_candles_low[i] * lotsize) / 9) > 1.0 * sell_wallet:
                        no_of_lots = z - 1
                        deployed_capital_2 = no_of_lots * ((all_candles_low[i] * lotsize) / 9)
                        break

                sell_wallet -= deployed_capital_2

                # StopLoss at candle LOW, GAP = 0.01%
                sl = (all_candles_low[i] + (all_candles_low[i] * 0.0028)).round(2)
                sl_array_sell = np.append(sl_array_sell, sl)     
                nishana = (all_candles_low[i] - (all_candles_low[i] * 0.006)).round(2)

                risk = sl - all_candles_low[i]
                reward = all_candles_low[i] - nishana
                rr_sell = (reward / risk).round(2) 
                net_rr_sell = np.append(net_rr_sell, rr_sell)

            # Exit at Target, SL or Day close
                for k in range(i, len(group)):
                    if all_candles_low[k] <= nishana:
                        exit_val = nishana
                        break
                    if all_candles_high[k] >= sl:
                        exit_val = sl
                        break
                    else:
                        exit_val = all_candles_close[-1]
                exit_sell = np.append(exit_sell, exit_val)
            
            # Append the earning
                earning_sell = deployed_capital_2 + ((all_candles_low[i] - exit_val) * lotsize * no_of_lots)
                sell_wallet = sell_wallet + earning_sell
                sell_earnings = np.append(sell_earnings, sell_wallet)


...
# N Conditions can be added here or seperately
# EMA20 and EMA50 will be used in the strategy later on.


backtesting_short_data = {
    "Date": dates_sell,
    "Time": time_sell,
    "Entry": entry_sell,
    "SL": sl_array_sell,
    "Exit": exit_sell,
    "RR": net_rr_sell,
    "P&L": None,
    "Earnings": sell_earnings,
    }

backtesting_sell_df = pd.DataFrame(backtesting_short_data)

In [148]:
backtesting_sell_df["P&L"] = (backtesting_sell_df["Exit"] - backtesting_sell_df["Entry"]).round(2)

backtesting_sell_df

,Date,Time,Entry,SL,Exit,RR,P&L,Earnings
0,2015-01-12,09:20:00,8257.95,8281.07,8281.07,2.14,23.12,975724.0
1,2015-02-04,09:40:00,8759.55,8784.08,8706.99,2.14,-52.56,1028284.0
2,2015-02-04,14:50:00,8761.75,8786.28,8709.18,2.14,-52.57,1083482.5
3,2015-02-20,14:35:00,8861.05,8885.86,8827.70,2.14,-33.35,1120167.5
4,2015-02-23,10:10:00,8841.00,8865.75,8787.95,2.14,-53.05,1178522.5
...,...,...,...,...,...,...,...,...
1276,2023-07-05,13:30:00,19355.20,19409.39,19409.39,2.14,54.19,12612537.5
1277,2023-07-10,12:20:00,19358.15,19412.35,19367.85,2.14,9.70,12566462.5
1278,2023-07-10,12:50:00,19377.10,19431.36,19367.85,2.14,-9.25,12610400.0
1279,2023-07-10,14:35:00,19375.40,19429.65,19367.85,2.14,-7.55,12646262.5


### 7. Getting Results.

In [144]:
total_trades_df = pd.concat([backtesting_buy_df, backtesting_sell_df], ignore_index = True)
total_trades_df["Net Earnings"] = None
total_trades_df["NP&L"] = None
total_trades_df.sort_values(by = "Date")

total_trades_df["Net Earnings"] = total_trades_df["Earnings"].cumsum()
total_trades_df["Total RR"] = total_trades_df["RR"].cumsum()

total_trades_df = total_trades_df[["Date", "Time", "Entry", "SL", "Exit", "RR", "Total RR"]]
total_trades_df

,Date,Time,Entry,SL,Exit,RR,Total RR
0,2015-02-10,12:30:00,8583.25,8558.36,8558.36,1.97,1.97
1,2015-02-18,11:15:00,8857.70,8832.01,8865.25,1.97,3.94
2,2015-02-19,15:15:00,8911.80,8885.96,8898.35,1.97,5.91
3,2015-02-28,15:15:00,8911.25,8885.41,8920.75,1.97,7.88
4,2015-05-15,10:05:00,8260.90,8236.94,8236.94,1.97,9.85
...,...,...,...,...,...,...,...
2851,2023-07-05,11:20:00,19367.85,19427.89,19415.30,1.97,5618.44
2852,2023-07-05,13:45:00,19367.05,19427.09,19415.30,1.97,5620.41
2853,2023-07-10,13:25:00,19379.25,19439.33,19367.85,1.97,5622.38
2854,2023-07-12,14:40:00,19443.05,19503.32,19384.75,1.97,5624.35


In [152]:
rr = (backtesting_sell_df['RR'].iloc[-1]).round(2)
total_trades = len(backtesting_sell_df)
positive_trades = (backtesting_sell_df[backtesting_sell_df["P&L"] > 0]["P&L"]).size

print("BACKTESTING SUMMARY [STRATEGY: CANDLE PATTERNS + PIVOT LEVELS]\n-------------------------------------------------------------------------------------")
print("* DATASET: NIFTY50 JAN 2015 - JUL 2023 | * TRADE CAPITAL: 10,00,000 INR             |")
print(f"* TIME FRAME: 5 Minutes                | * CAPITAL UTILIZATION PER TRADE: 100%      |")
print("* NIFTY FUTURES: INTRADAY              | * TRADING TIMELINE: 7 YEARS                |")
print("-------------------------------------------------------------------------------------")
print(f"* NO. OF TRADES EXECUTED: {total_trades}         | * 1st BUY STRATEGY: 2024000 INR = 102.4%   |")
print(f"* RR RATIO: {rr}:1                     | * 1st SELL STRATEGY: 7901806.5 INR = 690%  |")
print(f"* BEST ACCURACY: {round(((positive_trades / total_trades) * 100), 2)}%                | * 2nd SELL STRATEGY: 12954775 INR = 1200%  |")
print("-------------------------------------------------------------------------------------")


BACKTESTING SUMMARY [STRATEGY: CANDLE PATTERNS + PIVOT LEVELS]
-------------------------------------------------------------------------------------
* DATASET: NIFTY50 JAN 2015 - JUL 2023 | * TRADE CAPITAL: 10,00,000 INR             |
* TIME FRAME: 5 Minutes                | * CAPITAL UTILIZATION PER TRADE: 100%      |
* NIFTY FUTURES: INTRADAY              | * TRADING TIMELINE: 7 YEARS                |
-------------------------------------------------------------------------------------
* NO. OF TRADES EXECUTED: 1281         | * 1st BUY STRATEGY: 2024000 INR = 102.4%   |
* RR RATIO: 2.14:1                     | * 1st SELL STRATEGY: 7901806.5 INR = 690%  |
* BEST ACCURACY: 54.72%                | * 2nd SELL STRATEGY: 12954775 INR = 1200%  |
-------------------------------------------------------------------------------------


#### 2nd Selling strategy is giving <b>1200%</b> returns from 2015 to 2023.

### 8. Scope of Improvements.
- Use of EMA20 and EMA50.
- Creating better and more strategies.
- Automating search for best combinations of SL, Target and Candle-Stick values.
- Increasing efficiency and speed.
- Better structure.